In [11]:
from ase.atoms import Atoms
from ase.io import read, write

In [61]:
def checkHDistance(struc: Atoms):
    for a in struc:
        for b in struc:
            # print(b.index)
            if (a.index != b.index) and (a.symbol == "H") and (b.symbol == "H"):
                d = struc.get_distance(a.index, b.index)
                if d < 0.8:
                    return False
    else:
        return True

def checkHCage(struc: Atoms):
    pass

for i in range(300):
    struc = read(f"./test/229/step_1/POSCAR_{i+1}")
    if checkHDistance(struc):
        print(i)
    # 非常典型的一个结构 属于 及其不像笼型结构的化合物
    # struc.arrays['positions'][0]





[ 3.28092043 -4.24349296 -4.24349296]
[3.7622067  0.48128626 4.24349296]
[-0.48128626  3.7622067   0.        ]
[3.7622067  0.         0.48128626]
[-0.48128626 -4.24349296  3.7622067 ]
[-0.48128626  0.48128626  0.        ]
[ 3.7622067   3.7622067  -4.24349296]
[ 3.33466546e-16 -4.24349296e+00  4.24349296e+00]
[3.28092043e+00 0.00000000e+00 2.46519033e-32]
[3.7622067 0.        3.7622067]
[-0.48128626 -4.24349296  0.48128626]
[ 3.7622067   0.53380424 -3.70968872]
[-0.48128626  3.70968872  0.53380424]
[3.7622067  0.53380424 3.70968872]
[ 3.7622067  -0.53380424  3.70968872]
[ 0.05251798 -4.24349296 -3.70968872]
[0.05251798 4.24349296 3.70968872]
[3.22840245 0.         0.53380424]
[-1.01509051 -4.24349296  3.70968872]
[ 5.25179803e-02  5.33804245e-01 -6.16297582e-33]
[-1.01509051e+00  5.33804245e-01  3.88178420e-16]
[ 5.25179803e-02 -5.33804245e-01  3.88178420e-16]
[ 3.22840245  3.70968872 -4.24349296]
[-4.24349296  4.24349296 -4.24349296]
[-9.62572529e-01  0.00000000e+00  3.88178420e-16]
[-

KeyboardInterrupt: 

In [27]:
import itertools
import numpy as np
from ase.geometry import complete_cell
from ase.geometry.minkowski_reduction import minkowski_reduce
from ase.utils import pbc2pbc
from ase.cell import Cell

def translate_pretty(fractional, pbc):
    """Translates atoms such that fractional positions are minimized."""

    for i in range(3):
        if not pbc[i]:
            continue

        indices = np.argsort(fractional[:, i])
        sp = fractional[indices, i]

        widths = (np.roll(sp, 1) - sp) % 1.0
        fractional[:, i] -= sp[np.argmin(widths)]
        fractional[:, i] %= 1.0
    return fractional


def wrap_positions(positions, cell, pbc=True, center=(0.5, 0.5, 0.5),
                   pretty_translation=False, eps=1e-7):
    """Wrap positions to unit cell.

    Returns positions changed by a multiple of the unit cell vectors to
    fit inside the space spanned by these vectors.  See also the
    :meth:`ase.Atoms.wrap` method.

    Parameters:

    positions: float ndarray of shape (n, 3)
        Positions of the atoms
    cell: float ndarray of shape (3, 3)
        Unit cell vectors.
    pbc: one or 3 bool
        For each axis in the unit cell decides whether the positions
        will be moved along this axis.
    center: three float
        The positons in fractional coordinates that the new positions
        will be nearest possible to.
    pretty_translation: bool
        Translates atoms such that fractional coordinates are minimized.
    eps: float
        Small number to prevent slightly negative coordinates from being
        wrapped.

    Example:

    >>> from ase.geometry import wrap_positions
    >>> wrap_positions([[-0.1, 1.01, -0.5]],
    ...                [[1, 0, 0], [0, 1, 0], [0, 0, 4]],
    ...                pbc=[1, 1, 0])
    array([[ 0.9 ,  0.01, -0.5 ]])
    """

    if not hasattr(center, '__len__'):
        center = (center,) * 3

    pbc = pbc2pbc(pbc)
    shift = np.asarray(center) - 0.5 - eps

    # Don't change coordinates when pbc is False
    shift[np.logical_not(pbc)] = 0.0

    assert np.asarray(cell)[np.asarray(pbc)].any(axis=1).all(), (cell, pbc)

    cell = complete_cell(cell)
    fractional = np.linalg.solve(cell.T,
                                 np.asarray(positions).T).T - shift

    if pretty_translation:
        fractional = translate_pretty(fractional, pbc)
        shift = np.asarray(center) - 0.5
        shift[np.logical_not(pbc)] = 0.0
        fractional += shift
    else:
        for i, periodic in enumerate(pbc):
            if periodic:
                fractional[:, i] %= 1.0
                fractional[:, i] += shift[i]

    return np.dot(fractional, cell)

def naive_find_mic(v, cell):
    """Finds the minimum-image representation of vector(s) v.
    Safe to use for (pbc.all() and (norm(v_mic) < 0.5 * min(cell.lengths()))).
    Can otherwise fail for non-orthorhombic cells.
    Described in:
    W. Smith, "The Minimum Image Convention in Non-Cubic MD Cells", 1989,
    http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.57.1696."""
    f = Cell(cell).scaled_positions(v)
    f -= np.floor(f + 0.5)
    vmin = f @ cell
    vlen = np.linalg.norm(vmin, axis=1)
    return vmin, vlen


def general_find_mic(v, cell, pbc=True):
    """Finds the minimum-image representation of vector(s) v. Using the
    Minkowski reduction the algorithm is relatively slow but safe for any cell.
    """

    cell = complete_cell(cell)
    rcell, _ = minkowski_reduce(cell, pbc=pbc)
    positions = wrap_positions(v, rcell, pbc=pbc, eps=0)

    # In a Minkowski-reduced cell we only need to test nearest neighbors,
    # or "Voronoi-relevant" vectors. These are a subset of combinations of
    # [-1, 0, 1] of the reduced cell vectors.

    # Define ranges [-1, 0, 1] for periodic directions and [0] for aperiodic
    # directions.
    ranges = [np.arange(-1 * p, p + 1) for p in pbc]

    # Get Voronoi-relevant vectors.
    # Pre-pend (0, 0, 0) to resolve issue #772
    hkls = np.array([(0, 0, 0)] + list(itertools.product(*ranges)))
    vrvecs = hkls @ rcell

    # Map positions into neighbouring cells.
    x = positions + vrvecs[:, None]

    # Find minimum images
    lengths = np.linalg.norm(x, axis=2)
    indices = np.argmin(lengths, axis=0)
    vmin = x[indices, np.arange(len(positions)), :]
    vlen = lengths[indices, np.arange(len(positions))]
    return vmin, vlen


def find_mic(v, cell, pbc=True):
    """Finds the minimum-image representation of vector(s) v using either one
    of two find mic algorithms depending on the given cell, v and pbc."""

    cell = Cell(cell)
    pbc = cell.any(1) & pbc2pbc(pbc)
    dim = np.sum(pbc)
    v = np.asarray(v)
    single = v.ndim == 1
    v = np.atleast_2d(v)

    if dim > 0:
        naive_find_mic_is_safe = False
        if dim == 3:
            vmin, vlen = naive_find_mic(v, cell)
            # naive find mic is safe only for the following condition
            if (vlen < 0.5 * min(cell.lengths())).all():
                naive_find_mic_is_safe = True  # hence skip Minkowski reduction

        if not naive_find_mic_is_safe:
            vmin, vlen = general_find_mic(v, cell, pbc=pbc)
    else:
        vmin = v.copy()
        vlen = np.linalg.norm(vmin, axis=1)

    if single:
        return vmin[0], vlen[0]
    else:
        return vmin, vlen



def conditional_find_mic(vectors, cell, pbc):
    """Return list of vector arrays and corresponding list of vector lengths
    for a given list of vector arrays. The minimum image convention is applied
    if cell and pbc are set. Can be used like a simple version of get_distances.
    """
    if (cell is None) != (pbc is None):
        raise ValueError("cell or pbc must be both set or both be None")
    if cell is not None:
        mics = [find_mic(v, cell, pbc) for v in vectors]
        vectors, vector_lengths = zip(*mics)
    else:
        vector_lengths = np.linalg.norm(vectors, axis=2)
    return [np.asarray(v) for v in vectors], vector_lengths